# Top-down learning a GNN potential for coarse-grained water

In this notebook, we will learn a graph neural network (GNN) potential for
coarse-grained water from experimental data.
The coarse-grained particle will represent the oxygen atom of atomistic water.
Analogous to the diamond example, we will directly apply the DiffTRe library
without looking under its hood. We define all components necessary to
initialize DiffTRe: experimental data, simulation box, GNN potential with
prior, simulator, loss function and optimizer.

In [ ]:
from chemtrain import difftre
from chemtrain.jax_md_mod import io, custom_quantity, custom_space, \
    custom_energy, custom_simulator
from jax_md import space, partition, simulate
import optax
import jax.numpy as jnp
from jax import random, checkpoint
import numpy as np
from scipy import interpolate as sci_interpolate
import matplotlib.pyplot as plt
from functools import partial
import pickle
import warnings
warnings.filterwarnings('ignore')  # disable warnings about float64 usage

optimization_pickle_file_path = 'saved_models/CG_water_GNN.pkl'
long_traj_pickle_file_path = 'saved_optimization_results/CG_water_long_traj.pkl'

### Experimental observables

Let's start by defining target observables. For top-down coarse-graining,
we choose experimental data from the paper _Quantum Differences between
Heavy and Light Water_
[(Soper and Benmore, 2008)](https://doi.org/10.1103/PhysRevLett.101.065502).
Specifically, we want to match the oxygen-oxygen radial distribution function
(RDF), the oxygen-oxygen-oxygen angular distribution function (ADF), and a
pressure of 1 bar.

In [ ]:
pressure_conversion = 16.6054  # from kJ/mol nm^-3 to bar
pressure_target = 1. / pressure_conversion  # 1 bar in kJ / mol nm^3

# O-O RDF
rdf_bin_centers, rdf_bin_boundaries, sigma_RDF = \
    custom_quantity.rdf_discretization(RDF_cut=1.0)  # cut RDF at 1nm
reference_rdf = np.loadtxt('../examples/data/experimental/O_O_RDF.csv')
rdf_spline = sci_interpolate.interp1d(reference_rdf[:, 0],
                                      reference_rdf[:, 1],
                                      kind='cubic')
reference_rdf = rdf_spline(rdf_bin_centers)
rdf_struct = custom_quantity.RDFParams(reference_rdf,
                                       rdf_bin_centers,
                                       rdf_bin_boundaries,
                                       sigma_RDF)

# O-O-O ADF
adf_bin_centers, sigma_ADF = custom_quantity.adf_discretization(nbins=200)
reference_adf = np.loadtxt('../examples/data/experimental/O_O_O_ADF.csv')
adf_spline = sci_interpolate.interp1d(reference_adf[:, 0],
                                      reference_adf[:, 1],
                                      kind='cubic')
reference_adf = adf_spline(adf_bin_centers)
# only triplets within 0.318 nm are considered, consistent with experimental data
adf_struct = custom_quantity.ADFParams(reference_adf,
                                       adf_bin_centers,
                                       sigma_ADF,
                                       r_outer=0.318,
                                       r_inner=0.)

target_dict = {'rdf': rdf_struct,
               'adf': adf_struct,
               'pressure': pressure_target}

### Simulation setup

Next, we define the simulation parameters.
Temperature and density are already determined by the experiment.

In [ ]:
system_temperature = 296.15  # Kelvin = 23 deg. Celsius
Boltzmann_constant = 0.0083145107  # in kJ / mol K
kbT = system_temperature * Boltzmann_constant

file = '../examples/data/confs/Water_experimental_3nm.gro'  # 901 particles
R_init, box = io.load_box(file)  # initial configuration
N = R_init.shape[0]

mass = 18.0154  # in u: O + 2 * H
density = mass * N * 1.66054 / jnp.prod(box)
print('Model Density:', density, 'g/l. Experimental density: 997.87 g/l')

# simulation times in ps
time_step = 0.002
total_time = 70. 
t_equilib = 10.  # equilibration time: skip states before sampling observables
print_every = 0.1  # save state every 0.1 ps for computation of averages
timings_struct = custom_simulator.process_printouts(time_step, total_time,
                                                    t_equilib, print_every)

# define displacement and shift functions from box: 
# use periodic_general as we take derivatives wrt. box for pressure
box_tensor, scale_fn = custom_space.init_fractional_coordinates(box)
R_init = scale_fn(R_init)
displacement, shift = space.periodic_general(box_tensor)


### GNN Potential

Now we define the GNN as well as the prior potential. As our prior potential,
we choose the repulsive part of the Lennard-Jones (LJ) potential.
As the prior length scale $\sigma$, we simply take the length scale of the
LJ potential of the SPC water model $\sigma=0.3165$ nm and set the energy
scale to $\epsilon = 1$ kJ / mol - reduced compared to the LJ potential as
we neglect the attractive term.

Both potentials are combined by adding the computed potential energy values.
The `energy` function that sums both potentials is initialized by the
`energy_fn_template`. The template enables updating GNN weights during
optimization without compromising "jitability". You can define your own
custom `energy_fn_template` following the in- and output signature shown below.

In [ ]:
# define random seed for initialization of model and simulation
key = random.PRNGKey(0)
model_init_key, simuation_init_key = random.split(key, 2)

r_cut = 0.5

# create neighbor list: also works with cell list.
# num of neighbors can be increased, but does not change much
neighbor_fn = partition.neighbor_list(displacement, box[0], r_cut,
                                      dr_threshold=0.05,
                                      capacity_multiplier=1.5,
                                      fractional_coordinates=True)
nbrs_init = neighbor_fn(R_init)

prior_fn = custom_energy.\
    generic_repulsion_neighborlist(displacement,
                                   sigma=0.3165,
                                   epsilon=1.,
                                   exp=12,
                                   initialize_neighbor_list=False)

init_fn, GNN_energy = custom_energy.DimeNetPP_neighborlist(displacement,
                                                           R_init,
                                                           nbrs_init,
                                                           r_cut)
init_params = init_fn(model_init_key, R_init, neighbor=nbrs_init)

def energy_fn_template(energy_params):
    gnn_energy = partial(GNN_energy, energy_params)
    def energy(R, neighbor, **dynamic_kwargs):
        return gnn_energy(R, neighbor=neighbor, **dynamic_kwargs) + \
               prior_fn(R, neighbor=neighbor, **dynamic_kwargs)
    return energy

### Loss function

As some observables depend on the potential (e.g. the pressure in this case),
we need the `energy_fn_template` to define the loss function.
`DiffTRe_init` builds a mean-squared-error (MSE) loss from a `quantity_dict`.
Building the `quantity_dict` requires to define for each observable
$O_k(S_i, U_{\theta})$ a function to compute the respective
observable for each state $S_i$, the corresponding target value, and a scale
$\gamma$ defining the relative importance
of each observable in the loss (see examples below).

In [ ]:
quantity_dict = {}
if 'rdf' in target_dict:
    rdf_struct = target_dict['rdf']
    rdf_fn = custom_quantity.initialize_radial_distribution_fun(box_tensor,
                                                                displacement,
                                                                rdf_struct)
    rdf_dict = {'compute_fn': checkpoint(rdf_fn),
                'target': rdf_struct.reference_rdf, 'gamma': 1.}
    quantity_dict['rdf'] = rdf_dict
if 'adf' in target_dict:
    adf_struct = target_dict['adf']
    adf_fn = custom_quantity.\
        initialize_angle_distribution_neighborlist(displacement,
                                                   adf_struct,
                                                   R_init=R_init,
                                                   nbrs_init=nbrs_init)
    adf_target_dict = {'compute_fn': checkpoint(adf_fn),
                       'target': adf_struct.reference_adf, 'gamma': 1.}
    quantity_dict['adf'] = adf_target_dict
if 'pressure' in target_dict:
    pressure_fn = custom_quantity.init_pressure(energy_fn_template, box_tensor)
    pressure_target_dict = {'compute_fn': checkpoint(pressure_fn),
                            'target': target_dict['pressure'], 'gamma': 1.e-7}
    quantity_dict['pressure'] = pressure_target_dict

### Simulator

Now we define the simulator used to generate reference trajectories.
Similar to the `energy_fn_template`, the `simulator_template` takes the
current potential energy function and returns a corresponding NVT simulator.
This is achieved by simply "baking in" all constant simulation parameters via
`partial`. For this example, we select a Nose-Hoover thermostat with a chain
length of 5. With a given simulator, we generate the initial simulation state
from which the first trajectory is sampled.

In [ ]:
energy_fn_init = energy_fn_template(init_params)
simulator_template = partial(simulate.nvt_nose_hoover, shift_fn=shift,
                             dt=time_step, kT=kbT, chain_length=5,
                             chain_steps=1)
init, _ = simulator_template(energy_fn_init)
state = init(simuation_init_key, R_init, mass=mass, neighbor=nbrs_init)
init_sim_state = (state, nbrs_init)

### Optimizer

We use an Adam optimizer with exponential learning rate decay.

In [ ]:
num_updates = 300
initial_lr = 0.003
lr_schedule = optax.exponential_decay(-initial_lr, 200, 0.01)
optimizer = optax.chain(
    optax.scale_by_adam(0.1, 0.4),
    optax.scale_by_schedule(lr_schedule)
)


### Initialize DiffTRe

With all individual components defined, we can initialize the DiffTRe trainer.

In [ ]:
trainer = difftre.Trainer(init_params, quantity_dict, simulator_template,
                          energy_fn_template, neighbor_fn, init_sim_state,
                          timings_struct, optimizer, kbT)

### Learning the GNN potential

The optimization takes several hours. You can skip it and load the trained
model in the next cell.

In [ ]:
trainer.train(num_updates)

trainer.save_trainer(optimization_pickle_file_path)

In [ ]:
# load results if necessary
trainer = difftre.Trainer.load_trainer(optimization_pickle_file_path)

## Visualize training process

To assess convergence, we can visualize the loss and compute-time per update.
In the beginning, recomputation of the reference trajectory is necessary at
each update, due to large parameter updates. Later, with decreased step sizes,
reference trajectories can be re-used frequently, resulting in a significant
speed-up as only the reweighting step and the gradient need to be computed.

In [ ]:
fig, ax1 = plt.subplots()
ax1.set_xlabel('Update Step')
ax1.set_ylabel('MSE Loss')
ax1.semilogy(trainer.losses, color='#3c5488ff', label='Loss')
ax2 = ax1.twinx()
ax2.plot(trainer.update_times, label='Time per update', color='#4dbbd5ff')
ax2.set_ylabel('$t$ in s')
fig.legend(loc="upper right", bbox_to_anchor=(0.38,0.2),
           bbox_transform=ax1.transAxes)
plt.savefig('Train_history_CG_water.png')
plt.show()

Next, we visualize convergence of target observables.

In [ ]:
predicted_quantities = trainer.predictions
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(16, 4))

if 'rdf' in predicted_quantities[0]:
    rdf_series = [prediction_dict['rdf'] for prediction_dict in
                  predicted_quantities]

    ax1.set_xlabel('$r$ in nm')
    ax1.set_ylabel('RDF')
    ax1.plot(rdf_bin_centers, rdf_series[0], label='initial guess',
             color='#00a087ff')
    ax1.plot(rdf_bin_centers, rdf_series[-1], label='predicted',
             color='#3c5488ff')
    ax1.plot(rdf_bin_centers, reference_rdf, label='target', linestyle='--',
             color='k')
    ax1.legend()


if 'adf' in predicted_quantities[0]:
    adf_series = [prediction_dict['adf'] for prediction_dict in
                  predicted_quantities]

    ax2.set_xlabel(r'$\alpha$ in rad')
    ax2.set_ylabel('ADF')
    ax2.plot(adf_bin_centers, adf_series[0], label='initial guess',
             color='#00a087ff')
    ax2.plot(adf_bin_centers, adf_series[-1], label='predicted',
             color='#3c5488ff')
    ax2.plot(adf_bin_centers, reference_adf, label='target', linestyle='--',
             color='k')
    ax2.legend(loc=1)

if 'pressure' in predicted_quantities[0]:
    pressure_series = [prediction_dict['pressure'] for prediction_dict in
                       predicted_quantities]

    ax3.set_xlabel('Update Step')
    ax3.set_ylabel('$p$ in $\mathrm{kJ / mol nm^3}$')
    ax3.plot(pressure_series, label='predicted', color='#3c5488ff')
    ax3.axhline(y=pressure_target, linestyle='--', color='k', label='target')
    ax3.legend()
    print('Pressure final epoch:', pressure_series[-1], 'kJ / mol nm^3')

plt.savefig('Predicted_CG_water_properties.png')
plt.show()

## Validate results

Given that we've trained the model on rather short trajectories, let's
validate the model on a longer trajectory. This allows to detect if the
model overfitted to initial conditions or drifts away from the targets.

In [ ]:
total_time_long = 11000.  # 11 ns
t_equilib_long = 1000.  # 1 ns
long_trajectory_struct = custom_simulator.process_printouts(time_step,
                                                            total_time_long,
                                                            t_equilib_long,
                                                            print_every)

trajectory_generator = custom_simulator.trajectory_generator_init(simulator_template,
                                                                  energy_fn_template,
                                                                  neighbor_fn,
                                                                  long_trajectory_struct)
long_traj_state = trajectory_generator(trainer.params, init_sim_state)
quantity_traj = difftre.quantity_traj(long_traj_state, quantity_dict,
                                      neighbor_fn, trainer.params)

long_traj_rdf = jnp.mean(quantity_traj['rdf'], axis=0)
long_traj_adf = jnp.mean(quantity_traj['adf'], axis=0)
long_traj_press = jnp.mean(quantity_traj['pressure'], axis=0)

with open(long_traj_pickle_file_path, 'wb') as f:
    pickle.dump([long_traj_rdf, long_traj_adf, long_traj_press], f)

In [ ]:
# you can skip the long run above and just load results
long_traj_rdf, long_traj_adf, long_traj_press = \
    pickle.load(open(long_traj_pickle_file_path, "rb" ))

Comparing results from the long trajectory with our targets confirms that
training was successful.

In [ ]:
print('Prediced pressure:', long_traj_press, 'kJ / mol nm^3; Target:',
      pressure_target, 'kJ / mol nm^3')

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4), dpi=300)

ax1.set_xlabel('$r$ in nm')
ax1.set_ylabel('RDF')
ax1.plot(rdf_bin_centers, long_traj_rdf, label='predicted', color='#3c5488ff')
ax1.plot(rdf_bin_centers, reference_rdf, label='target', linestyle='--',
         color='k')
ax1.legend()

ax2.set_xlabel(r'$\alpha$ in rad')
ax2.set_ylabel('ADF')
ax2.plot(adf_bin_centers, long_traj_adf, label='predicted', color='#3c5488ff')
ax2.plot(adf_bin_centers, reference_adf, label='target', linestyle='--',
         color='k')
ax2.legend(loc=1)

plt.savefig('Predicted_water_properties_long_traj.png')
plt.show()


Additionally, we can verify the 3-body structure by computing the tetrahedral
order parameter $q$ and comparing it to the experiment.

In [ ]:
q_fn = custom_quantity.init_tetrahedral_order_parameter(displacement)
q_dict = {'compute_fn': q_fn}
quantity_dict = {'q': q_dict}

quantity_traj = difftre.quantity_traj(long_traj_state, quantity_dict,
                                      neighbor_fn, trainer.params)
q_average = jnp.mean(quantity_traj['q'], axis=0)

print('Tetrahedal order parameter Model: q =', q_average,
      '. Experiment: q = 0.576')